In [1]:
import pandas as pd
df=pd.read_csv('diabetes.csv')
df.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [2]:
from sklearn.preprocessing import StandardScaler
X=df.iloc[:, :8]
y=df.iloc[:, 8]

In [3]:
print(X.shape, y.shape)

(768, 8) (768,)


In [4]:
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.models import Sequential

Using TensorFlow backend.


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=10)
print(X_train.shape, X_test.shape)

(614, 8) (154, 8)


In [6]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [7]:
def create_model(layers, activation):
    
    model=Sequential()
    
    for i, nodes in enumerate(layers):
        
        if(i == 0):
            model.add(Dense(nodes, kernel_initializer='glorot_uniform', input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes, kernel_initializer='glorot_uniform'))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [9]:
model=KerasClassifier(build_fn=create_model, verbose=0)

In [10]:
layers=[[20], [40, 20], [50, 30], [45, 30, 15]]
# layers=[(20), (40, 20), (50, 30), (45, 30, 15)]
activations=['sigmoid', 'relu']
param_grid=dict(layers=layers, activation=activations, batch_size=[128, 256], epochs=[30])

In [11]:
grid=GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
#for GridSearchCV  there is param_grid paramter removing the param_distribution parameter

In [12]:
grid_result=grid.fit(X_train, y_train)

In [13]:
print(grid_result.best_score_, grid_result.best_params_)

0.7719869601998345 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [15]:
y_predict=grid.predict(X_test)
y_predict=y_predict>0.5

In [16]:
cm=confusion_matrix(y_predict, y_test)
cm

array([[80, 23],
       [15, 36]], dtype=int64)

In [17]:
score=accuracy_score(y_predict, y_test)

In [18]:
y_test[:10]

568    0
620    0
456    0
197    1
714    0
27     0
277    0
64     1
720    0
475    0
Name: Outcome, dtype: int64

In [19]:
y_predict[:10]

array([[ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True]])

In [20]:
score

0.7532467532467533